In [1]:
import numpy as np
import os
import gzip
import struct
import random
import time

import scipy.ndimage as nd
import PIL.Image
from IPython.display import clear_output, Image, display
from io import BytesIO

In [2]:
def loadlabels(filepath):
    with gzip.open(filepath, "rb") as f:
        _magic, n = struct.unpack(">ll", f.read(2*4))
        assert(_magic == 2049)
        return f.read(n)

def loadimages(filepath):
    with gzip.open(filepath, "rb") as f:
        _magic, n, sx, sy = struct.unpack(">llll", f.read(4*4))
        assert(_magic == 2051)
        images = []
        for _ in range(n):
            images.append(np.frombuffer(f.read(sx*sy), dtype=np.dtype("B")).astype(np.float32)/255.0)
        return ((sx, sy), images)

def load(labelsfp, imagesfp):
    labels = loadlabels(labelsfp)
    images = loadimages(imagesfp)
    assert(len(labels) == len(images[1]))
    return (images[0], list(zip(labels, images[1])))

def mkimg(data, size, fmt='jpeg'):
    data = np.uint8(np.clip((1 - data.reshape(size))*255, 0, 255))
    f = BytesIO()
    PIL.Image.fromarray(data).save(f, fmt)
    return Image(data=f.getvalue())

In [3]:
print("loading train set")
imgsize, trainset = load("data/train-labels-idx1-ubyte.gz", "data/train-images-idx3-ubyte.gz")

print("loading test set")
_imgsize, testset = load("data/t10k-labels-idx1-ubyte.gz", "data/t10k-images-idx3-ubyte.gz")

assert(imgsize == _imgsize)

loading train set
loading test set


In [4]:
indices = np.random.randint(0, len(trainset),  5)
for digit, image in [(trainset[i][0], mkimg(trainset[i][1], imgsize)) for i in indices]:
    display(digit, image)

2

6

9

6

6

In [5]:
class Net:
    def __init__(self, sizes, mag=0):
        self.sizes = sizes
        if mag == 0:
            self.weights = [np.zeros((sx, sy), dtype=np.float32) for sx, sy in zip(sizes[1:], sizes[:-1])]
            self.biases = [np.zeros(s, dtype=np.float32) for s in sizes]
        else:
            self.weights = [mag*np.random.randn(sx, sy) for sx, sy in zip(sizes[1:], sizes[:-1])]
            self.biases = [mag*np.random.randn(s) for s in sizes]

    def __iter__(self):
        return iter(self.weights + self.biases)

    def zcopy(self):
        return Net(self.sizes)

    def mkmem(self):
        return [[np.zeros_like(b) for _ in range(2)] for b in self.biases]

    def fill(self, value):
        for w in self.weights:
            w.fill(value)
        for b in self.biases:
            b.fill(value)

In [6]:
def act(x):
    return np.tanh(x)

def act_deriv(x):
        return 1/np.cosh(x)**2

def cost(a, y):
    d = a - y
    return np.dot(d, d)/2

def cost_deriv(a, y):
    return a - y

In [7]:
def feedforward(net, x, fmem=False):
    a = x + net.biases[0]
    if fmem:
        mem = net.mkmem()
        mem[0][0] = a
    else:
        mem = None

    for i, (w, b) in enumerate(zip(net.weights, net.biases[1:])):
        a = act(a)
        if fmem:
            mem[i][1] = a

        a = np.dot(w, a) + b
        if fmem:
            mem[i + 1][0] = a

    return (a, mem)

In [8]:
def backprop(net, a, y, mem, neterr):
    e = cost_deriv(a, y)
    zmem = list(zip(*mem))
    zlist = list(zip(
        net.weights, net.biases[1:],
        neterr.weights, neterr.biases[1:],
        zmem[0][1:], zmem[1][:-1]
    ))
    last = True
    for w, b, ew, eb, v, ap in reversed(zlist):
        if not last:
            e = e*act_deriv(v)
        eb += e
        ew += np.outer(e, ap)
        e = np.dot(e.transpose(), w)
        last = False
    neterr.biases[0] += e

In [9]:
def learn(net, neterr, rate, adagrad=None):
    if adagrad is None:
        for v, ev in zip(net, neterr):
            v -= rate*ev
    else:
        for v, ev, av in zip(net, neterr, adagrad):
            av += ev**2
            v -= rate*(ev/np.sqrt(av + 1e-8))

In [12]:
net = Net((imgsize[0]*imgsize[1], 30, 10), mag=1e-2)
neterr = net.zcopy()
adagrad = net.zcopy()

In [13]:
batchsize = 10
rate = 3e-1
for iepoch in range(10):
    print("epoch %d:" % iepoch)
    print("train:")
    totalcost = 0.0
    tstart = time.time()
    random.shuffle(trainset)
    for batch in [trainset[p:p+batchsize] for p in range(0, len(trainset), batchsize)]:
        for digit, img in batch:
            res = np.array([i == digit for i in range(10)])
            out, netmem = feedforward(net, img, fmem=True)
            totalcost += cost(out, res)
            backprop(net, out, res, netmem, neterr)
        learn(net, neterr, rate/batchsize, adagrad)
        neterr.fill(0)
    print("cost avg: %.4f" % (totalcost/len(trainset)))
    print("time elapsed: %.2f s" % (time.time() - tstart))

    print("test:")
    totalcost = 0.0
    hitcount = 0
    tstart = time.time()
    for digit, img in testset:
        res = np.array([i == digit for i in range(10)])
        out, _ = feedforward(net, img)
        totalcost += cost(out, res)
        hitcount += digit == np.argmax(out)
    print("cost avg: %.4f" % (totalcost/len(testset)))
    print("hit count: %d / %d" % (hitcount, len(testset)))
    print("time elapsed: %.2f s" % (time.time() - tstart))

print("done")

epoch 0:
train:
cost avg: 0.0871
time elapsed: 19.52 s
test:
cost avg: 0.0663
hit count: 9320 / 10000
time elapsed: 0.95 s
epoch 1:
train:
cost avg: 0.0657
time elapsed: 18.87 s
test:
cost avg: 0.0613
hit count: 9378 / 10000
time elapsed: 0.47 s
epoch 2:
train:
cost avg: 0.0615
time elapsed: 16.31 s
test:
cost avg: 0.0591
hit count: 9407 / 10000
time elapsed: 0.47 s
epoch 3:
train:
cost avg: 0.0593
time elapsed: 16.25 s
test:
cost avg: 0.0583
hit count: 9412 / 10000
time elapsed: 0.47 s
epoch 4:
train:
cost avg: 0.0577
time elapsed: 15.54 s
test:
cost avg: 0.0572
hit count: 9412 / 10000
time elapsed: 0.50 s
epoch 5:
train:
cost avg: 0.0565
time elapsed: 15.46 s
test:
cost avg: 0.0568
hit count: 9428 / 10000
time elapsed: 0.42 s
epoch 6:
train:
cost avg: 0.0555
time elapsed: 15.58 s
test:
cost avg: 0.0555
hit count: 9434 / 10000
time elapsed: 0.48 s
epoch 7:
train:
cost avg: 0.0547
time elapsed: 15.33 s
test:
cost avg: 0.0552
hit count: 9433 / 10000
time elapsed: 0.42 s
epoch 8:
train:


In [14]:
random.shuffle(testset)
for digit, img in testset[:5]:
        out, _ = feedforward(net, img)
        display("%s: %s"  % (digit, np.argmax(out)), mkimg(img, imgsize))

'8: 8'

'2: 2'

'2: 2'

'3: 3'

'7: 7'

In [17]:
rate = 1e-1
for digit in range(10):
    img = np.random.random(net.sizes[0])*1e-2
    res = np.array([i == digit for i in range(10)])
    for i in range(1000):
        out, mem = feedforward(net, img, fmem=True)
        cv = cost(out, res)
        neterr.fill(0)
        backprop(net, out, res, mem, neterr)
        img -= rate*neterr.biases[0]
        img = np.clip(img, 0, 1)
    display(np.argmax(out), cv, mkimg(img, imgsize))

0

0.00023544996053970207

1

6.793182579001262e-05

2

...

7.3839391687295e-05

8

0.008557040055692206

9

0.0005676274328840142